In [1]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab :(")

REP_PROJET = '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
REP_INPUT  = REP_PROJET + 'Input/'
REP_INTERMED  = REP_PROJET + 'Intermed/'
REP_OUTPUT  = REP_PROJET + 'Output/'

Mounted at /gdrive


In [15]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
!pwd

/gdrive/My Drive/Colab Notebooks/Dedoublement
/gdrive/My Drive/Colab Notebooks/Dedoublement


In [1]:
%%capture
! pip install -r requirements.txt

In [3]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from functools import partial

In [ ]:
%run "00_functions.ipynb"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
"""# Inputs"""

baseline = pd.read_pickle(REP_INTERMED + 'baseline.sav').drop([	'released_date'], axis=1)
cdandlp =  pd.read_pickle(REP_INTERMED + 'base_cdandlp.sav')
cdandlp['text']  = cdandlp['artiste']  + ' ' +  cdandlp['titre']


with open(REP_INTERMED + 'dict_words.pkl', 'rb') as f:
  dict_words = pickle.load(f)


analyzer = 'char'
ngram_range = (2,2)
X_train, vectorizer = reload_data_and_vectorizer(analyzer = analyzer, ngram_range=ngram_range)

In [ ]:
def knn(start , 
        end, 
        baseline = baseline, 
        dict_words=dict_words,
        df_cdandlp =cdandlp
        ) :

  df_sample= df_cdandlp[start : end]
  result = [ level1_knn_words(row_index, df_cdandlp, baseline, dict_words)  for row_index in range(start, end) ]
  
  with open(REP_INTERMED + str(end) + '_All_words_knn.pkl', 'wb') as  f1:
    pickle.dump(result, f1) 

In [ ]:
taille = len(cdandlp)
pas = int(taille/37)
#print(pas + 19080)

list_tmp = [ indices for indices in range(19080,  taille+1, pas) ]
[ (start, end) for start, end in zip(list_tmp,list_tmp[1:]) ]

[(19080, 20988),
 (20988, 22896),
 (22896, 24804),
 (24804, 26712),
 (26712, 28620),
 (28620, 30528),
 (30528, 32436),
 (32436, 34344),
 (34344, 36252),
 (36252, 38160),
 (38160, 40068),
 (40068, 41976),
 (41976, 43884),
 (43884, 45792),
 (45792, 47700),
 (47700, 49608),
 (49608, 51516),
 (51516, 53424),
 (53424, 55332),
 (55332, 57240),
 (57240, 59148),
 (59148, 61056),
 (61056, 62964),
 (62964, 64872),
 (64872, 66780),
 (66780, 68688),
 (68688, 70596)]

In [ ]:
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 4:06:54.475814


In [ ]:
taille = len(cdandlp)
pas = int(taille/37)
#13h 34
#16h 50
list_tmp = [ indices for indices in range(47700,  taille+1, pas) ]
start_time = datetime.now()
[ knn(start, end) for start, end in zip(list_tmp,list_tmp[1:]) ]
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
#21h 51 moins 22H 33

Duration: 6:02:54.361273


In [ ]:
knn(19080, 20988)

In [ ]:
# Duration: 4:06:54 ===> 55332
#11:57===> 18h 00 ===> 6h
# 02:45 8h 45 ======> 6h


In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
df_final = pd.read_pickle(REP_INTERMED + 'results_All_concat.pkl')

In [ ]:

#list_tmp = [ REP_INTERMED + str(indices) + '_All_words_knn.pkl'  for indices in range(pas,  taille+1, pas) ]
#list_tmp.remove('/gdrive/My Drive/Colab Notebooks/Dedoublement/Intermed/26712_All_words_knn.pkl')
#list_tmp.remove('/gdrive/My Drive/Colab Notebooks/Dedoublement/Intermed/55332_All_words_knn.pkl')


In [ ]:
## Concaténation et lecture des csv
dfs = [pd.concat(pd.read_pickle(path), ignore_index=True) for path in list(Path('Intermed').glob('*All_words_knn*'))]
df_final = pd.concat(dfs, ignore_index=True)
df_final.to_pickle(REP_INTERMED + 'results_All_concat.pkl')

In [ ]:
len(df_final['id_query'].astype(str).unique())

70596

In [ ]:
list(Path('Intermed').glob('*words_knn*'))

[PosixPath('Intermed/1908_All_words_knn.pkl'),
 PosixPath('Intermed/3816_All_words_knn.pkl'),
 PosixPath('Intermed/5724_All_words_knn.pkl'),
 PosixPath('Intermed/7632_All_words_knn.pkl'),
 PosixPath('Intermed/9540_All_words_knn.pkl'),
 PosixPath('Intermed/11448_All_words_knn.pkl'),
 PosixPath('Intermed/13356_All_words_knn.pkl'),
 PosixPath('Intermed/15264_All_words_knn.pkl'),
 PosixPath('Intermed/17172_All_words_knn.pkl'),
 PosixPath('Intermed/19080_All_words_knn.pkl'),
 PosixPath('Intermed/22896_All_words_knn.pkl'),
 PosixPath('Intermed/24804_All_words_knn.pkl'),
 PosixPath('Intermed/26712_All_words_knn.pkl'),
 PosixPath('Intermed/28620_All_words_knn.pkl'),
 PosixPath('Intermed/30528_All_words_knn.pkl'),
 PosixPath('Intermed/32436_All_words_knn.pkl'),
 PosixPath('Intermed/34344_All_words_knn.pkl'),
 PosixPath('Intermed/36252_All_words_knn.pkl'),
 PosixPath('Intermed/38160_All_words_knn.pkl'),
 PosixPath('Intermed/40068_All_words_knn.pkl'),
 PosixPath('Intermed/41976_All_words_knn.pkl'

In [ ]:
df_final.to_csv(REP_INTERMED + "resultats_knn_All.csv", index=False, encoding='utf-8-sig')

In [4]:
df_final = pd.read_csv(REP_INTERMED + "resultats_knn_All.csv", encoding='utf-8-sig')

In [7]:
len(df_final['id_query'].unique())

70596